In [3]:
import sys
from pathlib import Path
import pandas as pd

# Añade el directorio raíz del proyecto al sys.path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from rfh_testdatascience_1 import config

# Nombres de columnas (según adult.names)
column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

print(RAW_DATA_DIR)

# Leer los datos de entrenamiento
df_train = pd.read_csv("adult.data", header=None, names=column_names, na_values=" ?", skipinitialspace=True)

# Leer los datos de prueba (tiene una línea de encabezado no útil al inicio)
df_test = pd.read_csv("adult.test", header=0, names=column_names, na_values=" ?", skipinitialspace=True, comment='|')

# Limpiar target de prueba (tiene punto final en los valores)
df_test["income"] = df_test["income"].str.replace(".", "", regex=False)

# Mostrar primeras filas
print(df_train.head())
print(df_test.head())

ModuleNotFoundError: No module named 'rfh_testdatascience_1'